In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def warn(*args, **kwargs): pass
import warnings
warnings.warn = warn

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV

train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

RANDOM_STATE = 25

FileNotFoundError: [Errno 2] File b'../input/train.csv' does not exist: b'../input/train.csv'

In [2]:
from numpy.random import seed
seed(RANDOM_STATE)
from tensorflow import set_random_seed
set_random_seed(RANDOM_STATE)

In [3]:
# Swiss army knife function to organize the data

def encode(train, test):
    le = LabelEncoder().fit(train.label) 
    labels = le.transform(train.label)           # encode species strings
    classes = list(le.classes_)                    # save column names for submission
    test_ids = test.id                             # save test ids for submission
    
    train = train.drop(['label', 'id'], axis=1)  
    test = test.drop(['id'], axis=1)
    
    return train, labels, test, test_ids, classes

train, labels, test, test_ids, classes = encode(train, test)
train.head(5)

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,margin11,margin12,margin13,margin14,margin15,margin16,margin17,margin18,margin19,margin20,margin21,margin22,margin23,margin24,margin25,margin26,margin27,margin28,margin29,margin30,margin31,margin32,margin33,margin34,margin35,margin36,margin37,margin38,margin39,margin40,...,texture25,texture26,texture27,texture28,texture29,texture30,texture31,texture32,texture33,texture34,texture35,texture36,texture37,texture38,texture39,texture40,texture41,texture42,texture43,texture44,texture45,texture46,texture47,texture48,texture49,texture50,texture51,texture52,texture53,texture54,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
0,0.035156,0.105470,0.007812,0.000000,0.000000,0.175780,0.013672,0.000000,0.011719,0.025391,0.064453,0.000000,0.103520,0.001953,0.001953,0.0,0.031250,0.013672,0.003906,0.011719,0.005859,0.000000,0.0,0.000000,0.000000,0.003906,0.001953,0.000000,0.001953,0.000000,0.000000,0.000000,0.041016,0.001953,0.042969,0.064453,0.005859,0.009766,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.043945,0.051758,0.005859,0.040039,0.002930,0.000000,0.011719,0.005859,0.000000,0.0,0.080078,0.000000,0.002930,0.0,0.006836,0.000000,0.000000,0.092773,0.027344,0.063477,0.046875,0.005859,0.015625,0.000000,0.008789,0.000000,0.006836,0.012695,0.000000,0.066406,0.00000,0.012695,0.0,0.000000,0.000000,0.008789,0.002930
1,0.046875,0.115230,0.021484,0.000000,0.000000,0.132810,0.023438,0.005859,0.000000,0.033203,0.044922,0.000000,0.082031,0.000000,0.000000,0.0,0.023438,0.029297,0.003906,0.005859,0.000000,0.000000,0.0,0.000000,0.000000,0.009766,0.000000,0.001953,0.025391,0.001953,0.000000,0.000000,0.019531,0.001953,0.046875,0.064453,0.003906,0.009766,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.011719,0.059570,0.005859,0.094727,0.004883,0.000000,0.006836,0.009766,0.000000,0.0,0.035156,0.000000,0.002930,0.0,0.015625,0.000977,0.000000,0.039062,0.003906,0.047852,0.086914,0.005859,0.053711,0.000000,0.012695,0.000977,0.053711,0.019531,0.000000,0.013672,0.00000,0.012695,0.0,0.000000,0.000977,0.009766,0.000000
2,0.064453,0.123050,0.009766,0.003906,0.000000,0.085938,0.013672,0.000000,0.011719,0.019531,0.050781,0.001953,0.111330,0.001953,0.001953,0.0,0.009766,0.019531,0.007812,0.007812,0.001953,0.000000,0.0,0.000000,0.001953,0.007812,0.001953,0.000000,0.021484,0.001953,0.000000,0.000000,0.033203,0.003906,0.033203,0.068359,0.009766,0.009766,0.000000,0.000000,...,0.000000,0.000000,0.002930,0.000000,0.006836,0.001953,0.043945,0.000000,0.000000,0.050781,0.005859,0.000000,0.0,0.062500,0.000000,0.000977,0.0,0.000000,0.001953,0.012695,0.055664,0.048828,0.031250,0.039062,0.000977,0.000000,0.000000,0.000000,0.000000,0.006836,0.127930,0.000000,0.054688,0.00000,0.004883,0.0,0.000000,0.000000,0.000000,0.005859
3,0.000000,0.000000,0.037109,0.050781,0.003906,0.000000,0.003906,0.000000,0.048828,0.003906,0.001953,0.005859,0.007812,0.031250,0.005859,0.0,0.000000,0.009766,0.031250,0.003906,0.052734,0.005859,0.0,0.001953,0.011719,0.007812,0.017578,0.023438,0.099609,0.009766,0.011719,0.001953,0.003906,0.000000,0.000000,0.009766,0.017578,0.056641,0.005859,0.001953,...,0.007812,0.000977,0.008789,0.030273,0.021484,0.029297,0.019531,0.064453,0.000000,0.019531,0.023438,0.051758,0.0,0.009766,0.009766,0.003906,0.0,0.022461,0.011719,0.001953,0.002930,0.017578,0.019531,0.074219,0.014648,0.017578,0.021484,0.019531,0.002930,0.005859,0.038086,0.025391,0.009766,0.00293,0.021484,0.0,0.037109,0.006836,0.002930,0.036133
4,0.031250,0.029297,0.080078,0.001953,0.011719,0.076172,0.039062,0.000000,0.000000,0.031250,0.013672,0.005859,0.050781,0.000000,0.025391,0.0,0.025391,0.013672,0.000000,0.013672,0.001953,0.000000,0.0,0.000000,0.000000,0.007812,0.000000,0.003906,0.019531,0.015625,0.000000,0.000000,0.009766,0.000000,0.033203,0.031250,0.042969,0.033203,0.017578,0.005859,...,0.005859,0.000977,0.011719,0.008789,0.008789,0.003906,0.004883,0.000000,0.000977,0.07031

In [8]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=RANDOM_STATE)

for train_index, test_index in sss.split(train.values, labels):
    X_train, X_test = train.values[train_index], train.values[test_index]
    y_train, y_test = labels[train_index], labels[test_index]

In [5]:
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import GaussianNB, BernoulliNB 
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.linear_model import LogisticRegression
from keras.applications.xception import Xception

Using TensorFlow backend.


In [9]:
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)

scaler = StandardScaler().fit(X_test)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= KNeighborsClassifier(3)
clf.fit(X_train, y_train)
name = clf.__class__.__name__
    
print(name)
    
print('Results')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= GaussianProcessClassifier()
clf.fit(X_train, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= GaussianNB()
clf.fit(X_train, y_train)
name = clf.__class__.__name__

print(name)

print('Results')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= BernoulliNB()
clf.fit(X_train, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
param_grid = {'tol': [0.000001, 0.00001, 0.0001, 0.001]}
clf= MLPClassifier(random_state=RANDOM_STATE)
grid_search = GridSearchCV(clf, param_grid, cv=sss)
grid_search.fit(X_train, y_train)

print ('Best estimator:\n{}'.format(grid_search.best_estimator_))

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=25, shuffle=True, solver='adam', tol=1e-06,
       validation_fraction=0.1, verbose=False, warm_start=False)
clf.fit(X_train, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= SGDClassifier(loss="log", penalty="l2", max_iter=5)
clf.fit(X_train, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= SVC(kernel="rbf", C=0.025, probability=True)
clf.fit(X_train, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
param_grid = {'gamma': [ 1,10,100,1000],
              'tol': [0.000001, 0.00001, 0.0001, 0.001]}
clf = NuSVC(kernel="rbf",probability=True,random_state=RANDOM_STATE)
grid_search = GridSearchCV(clf, param_grid, cv=sss)
grid_search.fit(X_train, y_train)

print ('Best estimator:\n{}'.format(grid_search.best_estimator_))

In [ ]:
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= NuSVC(cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma=1, kernel='rbf',
   max_iter=-1, nu=0.5, probability=True, random_state=25, shrinking=True,
   tol=1e-06, verbose=False)
clf.fit(X_train, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
param_grid = {'C': [ 10,100,1000, 10000],
              'tol': [0.000001, 0.00001, 0.0001, 0.001]}
clf = LogisticRegression(solver='newton-cg',random_state = RANDOM_STATE)
grid_search = GridSearchCV(clf, param_grid, cv=sss)
grid_search.fit(X_train, y_train)

print ('Best estimator:\n{}'.format(grid_search.best_estimator_))

In [ ]:
#score = 0.6109
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=25, solver='newton-cg',
          tol=0.001, verbose=0, warm_start=False)
clf.fit(X_train, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

In [ ]:
param_grid = {'C': [ 10,100,1000, 10000],
              'tol': [0.000001, 0.00001, 0.0001, 0.001]}
clf = LogisticRegression(solver='newton-cg',random_state = RANDOM_STATE)
grid_search = GridSearchCV(clf, param_grid, cv=sss)
grid_search.fit(X_train_scaled, y_train)

print ('Best estimator:\n{}'.format(grid_search.best_estimator_))

In [26]:
#score=0.00171
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=25, solver='newton-cg',
          tol=0.001, verbose=0, warm_start=False)
clf.fit(X_train_scaled, y_train)
name = clf.__class__.__name__

print(name)

print('Results')
train_predictions = grid_search.predict(X_test_scaled)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = grid_search.predict_proba(X_test_scaled)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

LogisticRegression
Results
Accuracy: 97.1223%
Log Loss: 0.12242191399839837


In [25]:
#score=0.00219
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=25, solver='newton-cg',
          tol=1e-06, verbose=0, warm_start=False)
clf.fit(X_train_scaled, y_train)
name = clf.__class__.__name__

print(name)

print('Results')
train_predictions = clf.predict(X_test_scaled)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test_scaled)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

LogisticRegression
Results
Accuracy: 98.5612%
Log Loss: 0.0676512510613953


In [ ]:
#score=0.00219
## Importing sklearn libraries

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Keras Libraries for Neural Networks

from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping
from keras import optimizers

## Read data from the CSV file

data = pd.read_csv('../input/train.csv')
parent_data = data.copy()    ## Always a good idea to keep a copy of original data
ID = data.pop('id')

y = data.pop('label')
y = LabelEncoder().fit(y).transform(y)
print(y.shape)

X = scaler.transform(data)
print(X.shape)

y_cat = to_categorical(y)
print(y_cat.shape)


model = Sequential()
model.add(Dense(1024,input_dim=192,  init='uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(516, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(99, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(),metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_loss', patience=280)
history = model.fit(X,y_cat,batch_size=192,
                    nb_epoch=800 ,verbose=0, validation_split=0.1, callbacks=[early_stopping])
                    


print('val_acc: ',max(history.history['val_acc']))
print('val_loss: ',min(history.history['val_loss']))
print('train_acc: ',max(history.history['acc']))
print('train_loss: ',min(history.history['loss']))

print()
print("train/val loss ratio: ", min(history.history['loss'])/min(history.history['val_loss']))

test = pd.read_csv('../input/test.csv')
index = test.pop('id')
test = scaler.transform(test)
submission = model.predict_proba(test)

In [ ]:
#score=0.04926
## Importing standard libraries

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Importing sklearn libraries

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

## Keras Libraries for Neural Networks

from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping

Y_train = to_categorical(y_train)
print(Y_train.shape)


model = Sequential()
model.add(Dense(2048,input_dim=192, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(99, activation='softmax'))

model.compile(loss='categorical_crossentropy',optimizer=optimizers.Adam(),metrics=['acc'])

early_stopping = EarlyStopping(monitor='val_loss', min_delta=0,patience=250, verbose=1, mode='min',
                               baseline=None, restore_best_weights=True)
history = model.fit(X_train_scaled,Y_train,batch_size=25,
                    nb_epoch=500 ,verbose=1, validation_split=0.1,callbacks=[early_stopping])
                    


print('val_acc: ',max(history.history['val_acc']))
print('val_loss: ',min(history.history['val_loss']))
print('train_acc: ',max(history.history['acc']))
print('train_loss: ',min(history.history['loss']))

print()
print("train/val loss ratio: ", min(history.history['loss'])/min(history.history['val_loss']))

submission = model.predict_proba(test)

In [ ]:
submission = pd.DataFrame(submission,columns=classes)

submission.insert(0, 'id', test_ids)
submission.reset_index()

submission.to_csv('tugas3_0102516002.csv', index = True)
submission.tail()

In [48]:
#score=0.00132
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= LogisticRegression(C=1000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=25, solver='newton-cg',
          tol=0.001, verbose=0, warm_start=False)
clf.fit(X_train_scaled, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test_scaled)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test_scaled)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

LogisticRegression
Results
Accuracy: 99.2806%
Log Loss: 0.02671106090401736


In [90]:
#score=0.00143
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= LogisticRegression(C=10000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=25, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)
clf.fit(X_train_scaled, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test_scaled)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test_scaled)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

LogisticRegression
Results
Accuracy: 99.2806%
Log Loss: 0.01924348905614107


In [105]:
#score=0.00140
# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss"]
log = pd.DataFrame(columns=log_cols)

clf= LogisticRegression(C=100000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=25, solver='lbfgs',
          tol=0.00001, verbose=0, warm_start=False)
clf.fit(X_train_scaled, y_train)
name = clf.__class__.__name__

print(name)
    
print('Results')
train_predictions = clf.predict(X_test_scaled)
acc = accuracy_score(y_test, train_predictions)
print("Accuracy: {:.4%}".format(acc))
    
train_predictions = clf.predict_proba(X_test_scaled)
ll = log_loss(y_test, train_predictions)
print("Log Loss: {}".format(ll))
    
log_entry = pd.DataFrame([[name, acc*100, ll]], columns=log_cols)
log = log.append(log_entry)

LogisticRegression
Results
Accuracy: 100.0000%
Log Loss: 0.013790283404440528


In [106]:
# Predict Test Set
favorite_clf = LogisticRegression(C=100000, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=None, penalty='l2', random_state=25, solver='lbfgs',
          tol=0.00001, verbose=0, warm_start=False)
favorite_clf.fit(X_train_scaled, y_train)
test_scaled = scaler.transform(test)
test_predictions = favorite_clf.predict_proba(test_scaled)

# Format DataFrame
submission = pd.DataFrame(test_predictions, columns=classes)
submission.insert(0, 'id', test_ids)
submission.reset_index()

# Export Submission
submission.to_csv('tugas3_0102516002.csv', index = True)
submission.tail()

,id,Acer_Capillipes,Acer_Circinatum,Acer_Mono,Acer_Opalus,Acer_Palmatum,Acer_Pictum,Acer_Platanoids,Acer_Rubrum,Acer_Rufinerve,Acer_Saccharinum,Alnus_Cordata,Alnus_Maximowiczii,Alnus_Rubra,Alnus_Sieboldiana,Alnus_Viridis,Arundinaria_Simonii,Betula_Austrosinensis,Betula_Pendula,Callicarpa_Bodinieri,Castanea_Sativa,Celtis_Koraiensis,Cercis_Siliquastrum,Cornus_Chinensis,Cornus_Controversa,Cornus_Macrophylla,Cotinus_Coggygria,Crataegus_Monogyna,Cytisus_Battandieri,Eucalyptus_Glaucescens,Eucalyptus_Neglecta,Eucalyptus_Urnigera,Fagus_Sylvatica,Ginkgo_Biloba,Ilex_Aquifolium,Ilex_Cornuta,Liquidambar_Styraciflua,Liriodendron_Tulipifera,Lithocarpus_Cleistocarpus,Lithocarpus_Edulis,...,Quercus_Coccinea,Quercus_Crassifolia,Quercus_Crassipes,Quercus_Dolicholepis,Quercus_Ellipsoidalis,Quercus_Greggii,Quercus_Hartwissiana,Quercus_Ilex,Quercus_Imbricaria,Quercus_Infectoria_sub,Quercus_Kewensis,Quercus_Nigra,Quercus_Palustris,Quercus_Phellos,Quercus_Phillyraeoides,Quercus_Pontica,Quercus_Pubescens,Quercus_Pyrenaica,Quercus_Rhysophylla,Quercus_Rubra,Quercus_Semecarpifolia,Quercus_Shumardii,Quercus_Suber,Quercus_Texana,Quercus_Trojana,Quercus_Variabilis,Quercus_Vulcanica,Quercus_x_Hispanica,Quercus_x_Turneri,Rhododendron_x_Russellianum,Salix_Fragilis,Salix_Intergra,Sorbus_Aria,Tilia_Oliveri,Tilia_Platyphyllos,Tilia_Tomentosa,Ulmus_Bergmanniana,Viburnum_Tinus,Viburnum_x_Rhytidophylloides,Zelkova_Serrata
292,632,3.261130e-11,8.975804e-15,1.437572e-10,2.142681e-10,7.557601e-16,6.862260e-16,1.866919e-09,3.093254e-13,4.377176e-11,3.310273e-17,4.153903e-15,1.303029e-13,2.343242e-13,4.390145e-17,8.065884e-15,2.269314e-15,2.539097e-15,1.460674e-08,1.062461e-12,1.757986e-10,1.385112e-15,1.881097e-19,2.548388e-19,5.705811e-17,1.716480e-15,3.525389e-17,9.513970e-11,3.518191e-09,6.277038e-18,2.248806e-15,4.158242e-14,1.297320e-10,4.975919e-12,4.570240e-17,8.239220e-13,5.421293e-17,9.633607e-09,6.503078e-11,1.230011e-12,...,6.296901e-10,4.063182e-09,3.111627e-15,2.840018e-13,6.115324e-11,1.321327e-18,1.148501e-15,6.730210e-14,6.729314e-12,8.684416e-14,1.935967e-09,1.292746e-17,2.712781e-13,3.505124e-16,1.701593e-10,3.201889e-18,8.775846e-14,2.493123e-09,4.186811e-15,4.117503e-16,3.633030e-11,9.518896e-17,1.099445e-07,6.048138e-08,1.852910e-14,1.732767e-10,6.323031e-09,1.697497e-12,1.092883e-09,2.859195e-17,2.094220e-07,4.037141e-10,7.850255e-14,7.210749e-13,6.474528e-17,1.206736e-09,2.611089e-08,7.938953e-13,1.755170e-16,1.275679e-16
293,1446,6.064140e-11,1.192330e-14,4.514459e-15,8.597442e-13,1.237646e-13,2.524007e-10,5.566480e-11,8.430306e-15,2.970209e-11,5.619148e-12,1.909569e-13,5.409840e-13,8.624439e-13,1.921335e-12,7.598427e-12,1.193862e-13,2.803221e-13,4.827575e-14,1.355889e-11,2.081161e-12,5.254746e-13,2.306077e-18,1.072200e-14,4.563768e-13,1.804681e-11,1.646594e-14,4.682022e-12,1.560616e-16,1.704399e-11,6.484059e-16,1.246500e-13,8.364357e-17,9.698672e-16,6.659076e-13,1.048501e-17,1.340954e-11,2.963649e-14,8.472943e-13,4.090438e-15,...,3.615438e-09,1.271000e-14,4.413259e-14,8.120550e-16,2.961938e-09,6.353206e-18,4.309357e-12,3.157042e-17,2.166694e-15,4.950509e-13,8.890376e-11,2.157640e-09,1.000000e+00,1.781295e-13,1.369131e-12,6.817049e-15,1.966568e-15,6.674452e-12,1.632606e-13,9.686631e-12,2.920977e-14,1.393148e-08,5.049525e-15,5.779273e-10,2.836923e-16,1.251700e-13,2.304191e-12,1.403168e-12,1.282808e-13,1.787532e-15,1.389529e-12,1.237847e-13,2.995094e-13,2.850506e-15,5.228290e-13,7.331052e-17,6.940218e-13,1.781027e-13,6.163629e-13,1.139315e-16
294,37,4.340638e-20,2.284473e-14,3.466255e-15,3.553605e-17,7.293414e-16,1.395770e-10,6.900056e-13,7.102265e-15,6.841958e-15,6.075262e-17,1.575866e-19,2.331373e-15,1.527255e-17,1.890531e-15,3.826149e-18,1.000000e+00,6.529371e-19,1.807131e-15,2.704668e-17,4.501359e-17,3.867825e-19,1.119373e-16,6.235367e-13,3.501895e-15,2.832656e-15,7.046263e-17,4.160535e-19,3.424362e-15,9.259372e-11,3.390484e-13,8.308372e-15,1.417392e-17,6.341387e-15,1.254855e-12,4.040019e-18,1.223365e-14,1.020921e-15,3.416421e-15,6.668435e-16,

In [107]:
from IPython.display import HTML
import base64

def create_download_link(df, title = "Download CSV file", filename = "tugas3_0102516002.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe
create_download_link(submission)